# BUSCA

In [ ]:
import csv
import xml.etree.ElementTree as ET
from collections import Counter
import csv
import ast
import math

##Tratamento das configurações:
*Modelo, consultas e resultados*

In [ ]:
with open('/content/drive/MyDrive/BMT/T2/configs/BUSCA.cfg' , 'r') as f:
    for line in f:
        if line.startswith('MODELO='):
            model = line.split('=')[1].strip()
        elif line.startswith('CONSULTAS='):
            consultas = line.split('=')[1].strip()
        elif line.startswith('RESULTADOS='):
            results = line.split('=')[1].strip()

### Lendo e indexando no dicionário o modelo

In [ ]:
dic = {}
with open(model) as model_file:
    model_reader = csv.reader(model_file, delimiter=";")
    next(model_reader)
    
    for l in model_reader:
        if l[0] not in dic.keys():
            dic[l[0]] = {int(l[1]): float(l[2])}
        else:
            dic[l[0]][int(l[1])] = float(l[2])

###Ajustando o arquivo para armazenar os resultados

In [ ]:
with open(results, "w", newline='') as result_file:
    writer = csv.writer(result_file, delimiter=";")
    writer.writerow(["QueryNumber", "[DocRanking, DocNumber, Similarity]"])

In [ ]:
##percorrendo as consultas para calcular os pesos
with open(consultas) as query_file:
    query_reader = csv.reader(query_file, delimiter=";")
    next(query_reader)
    query_num = 0
    result_lines = 0
        
    for q in query_reader:
        query_dict = {}
        words = q[1].split()
        words = [w for w in words if len(w) >= 2]
        query_vec = Counter(words)
        
        for qv in query_vec:
            if qv in dic.keys():
                current_dict = dic[qv]
                weight_list = []
                for key in current_dict:
                    weight_list.append(current_dict[key])
                    if key not in query_dict:
                        query_dict[key] = current_dict[key] * query_vec[qv]
                    else:
                        query_dict[key] += current_dict[key] * query_vec[qv]
        
        query_num += 1

        #ordenando
        sorted_values = sorted(query_dict.items(), key=lambda item: item[1], reverse=True)
        

###Escrevendo o arquivo de resultados
        with open(results, "a", newline='') as result_file:
            result_writer = csv.writer(result_file, delimiter=";")
            for i, e in enumerate(sorted_values):
                li = [i, e[0], e[1]]
                result_writer.writerow([q[0], li])
                result_lines += 1

##Invert lists

In [ ]:
def parse_xml_file(xml_file):
        tree = ET.parse(xml_file)
        root = tree.getroot()

        docs = []
        for record in root.findall("./RECORD"):
            doc_id = record.find("./RECORDNUM").text
            abstract = record.find("./ABSTRACT")
            if abstract is None:
                abstract = record.find("./EXTRACT")
            if abstract is not None:
                doc_text = abstract.text.strip()
                doc_text = doc_text.translate(str.maketrans("", "", string.punctuation)).upper()
                docs.append((doc_id, doc_text))

        return docs

In [ ]:
def create_inverted_index(documents):
    inverted_index = {}
    for doc_id, doc_text in documents:
        for word in doc_text.split():
            if word not in inverted_index:
                inverted_index[word] = [int(doc_id)]
            else:
                inverted_index[word].append(int(doc_id))

    return inverted_index

In [ ]:
def write_inverted_index_to_file(inverted_index, output_file):
    with open(output_file, "w", newline='') as f:
        writer = csv.writer(f, delimiter=";")
        writer.writerow(["Word", "Documents"])
        for word, docs in inverted_index.items():
            writer.writerow([word, ",".join(map(str, docs))])

In [ ]:
import os
import xml.etree.ElementTree as ET
import string
import csv

input_files = []
output_file=None
with open('/content/drive/MyDrive/BMT/T2/configs/GLI.cfg' , 'r') as f:
    for line in f:
        line = line.strip()
        if line.startswith("LEIA="):
            input_files.append(line[len("LEIA="):])
        elif line.startswith("ESCREVA="):
            output_file = line[len("ESCREVA="):]  

documents = []
for input_file in input_files:
    documents += parse_xml_file(input_file)
inverted_index = create_inverted_index(documents)
write_inverted_index_to_file(inverted_index, output_file)

#INDEX

In [ ]:
with open('/content/drive/MyDrive/BMT/T2/configs/INDEX.cfg' , 'r') as f:
    for line in f:
        if line.startswith('LEIA='):
            leia = line.split('=')[1].strip()
        elif line.startswith('ESCREVA='):
            escreva = line.split('=')[1].strip()
        elif line.startswith('FREQUENCIA='):
            f = line.split('=')[1].strip()

In [ ]:
qp = []
voc = 0

with open(leia, newline='\n') as csv_file:
    reader = csv.reader(csv_file, delimiter=";")
    next(reader)
    for l in reader:
        li = ast.literal_eval(l[1])
        voc += 1

        for e in li:
            while e > len(qp):
                qp.append(0)
            qp[e - 1] += 1

qd = len(qp)

with open(leia, newline='\n') as read_file, \
    open(escreva, "w", newline='') as write_file:
    reader = csv.reader(read_file, delimiter=";")
    next(reader)
    
    writer = csv.writer(write_file, delimiter=";")
    writer.writerow(["Word", "DocNumber", "Weight"])

    lines_read = 0
    lines_written = 0
    for l in reader:
        lines_read += 1
        li = ast.literal_eval(l[1])
        c = Counter(li)
        
        for doc in c:
            if f == "absoluta":
                weight = c[doc] * math.log(qd/len(set(li)) + 1)
            elif f == "relativa":
                weight = (c[doc]/qp[doc - 1]) * math.log(qd/len(set(li)) + 1)
            
            lines_written += 1
            writer.writerow([l[0], doc, weight])

#PC

In [ ]:
with open('/content/drive/MyDrive/BMT/T2/configs/PC.cfg' , 'r') as f:
    for line in f:
        if line.startswith('LEIA='):
            xml_file = line.split('=')[1].strip()
        elif line.startswith('CONSULTAS='):
            consultas_file = line.split('=')[1].strip()
        elif line.startswith('ESPERADOS='):
            results_file = line.split('=')[1].strip()

In [ ]:
import xml.etree.ElementTree as ET
import csv

def pc(xml_file, consultas_file, results_file):
    # Faz a leitura do arquivo XML
    tree = ET.parse(xml_file)
    root = tree.getroot()

    consultas = []
    expected_results = []

    for query in root.iter('QUERY'):
        query_number = query.find('QueryNumber').text
        query_text = query.find('QueryText').text.upper().replace(';', '')
        consultas.append([query_number, query_text])

        for item in query.findall('Records/Item'):
            doc_number = item.text
            score = item.get('score')
            doc_votes = len(score.replace('0', ''))
            expected_results.append([query_number, doc_number, doc_votes])

    with open(consultas_file, 'w', newline='') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow(['QueryNumber', 'QueryText'])
        writer.writerow(consultas)

    with open(results_file, 'w', newline='') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow(['QueryNumber', 'DocNumber', 'DocVotes'])
        writer.writerow(expected_results)

pc(xml_file, consultas_file, results_file)
